In [4]:
pip install -q torch torchvision onnxruntime-gpu tqdm onnx

In [2]:
import torch, torchvision, time, onnx, onnxruntime, numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


ДЗ (1 ⭐) Выведите torch.cuda.get_device_name(0) и torch.cuda.get_device_properties(0).total_memory в Markdown-таблицу. Сохраните вывод в docs/gpu_info.md.

In [5]:
import torch
import os

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory
else:
    gpu_name = "CUDA not available"
    gpu_mem = 0

# Markdown таблица
markdown = (
    "| Property | Value |\n"
    "|----------|-------|\n"
    f"| GPU Name | {gpu_name} |\n"
    f"| Total Memory (bytes) | {gpu_mem} |\n"
)

# Создаем папку docs/
os.makedirs("docs", exist_ok=True)

# Пишем в файл
with open("docs/gpu_info.md", "w") as f:
    f.write(markdown)
